In [66]:
import pandas as pd
import os

In [67]:
os.chdir(r'C:\SML_Projects\SML_CVE_type_cwe_predict')

In [68]:
from src.preprocessing import Preprocessing

In [69]:
df = pd.read_csv("data/filled/filled_dataset.csv")

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102963 entries, 0 to 102962
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   cve_id        102963 non-null  object
 1   description   102963 non-null  object
 2   cvss_score    102963 non-null  object
 3   cwe           102963 non-null  object
 4   vendor        102963 non-null  object
 5   product       102963 non-null  object
 6   publish_date  102963 non-null  object
 7   type          102963 non-null  object
dtypes: object(8)
memory usage: 6.3+ MB


In [71]:
preprocessing = Preprocessing(df, target=['type', 'cvss_score'])

In [72]:
df = preprocessing.encode(include_targets=True).scale().get_dataset()

In [73]:
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10296 entries, 0 to 10295
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cve_id        10296 non-null  float64
 1   description   10296 non-null  float64
 2   cvss_score    10296 non-null  float64
 3   cwe           10296 non-null  float64
 4   vendor        10296 non-null  float64
 5   product       10296 non-null  float64
 6   publish_date  10296 non-null  float64
 7   type          10296 non-null  float64
dtypes: float64(8)
memory usage: 643.6 KB


In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier

In [76]:
x = df.drop(['type', 'cvss_score'], axis=1)   
y = df[['type', 'cvss_score']] 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [77]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Logistic Regression

In [78]:
lr = LogisticRegression()
multi_lr = MultiOutputClassifier(lr)

multi_lr.fit(x_train, y_train)
y_pred = multi_lr.predict(x_test)

lr_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
lr_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

lr_scores_type = cross_val_score(lr, x, y['type'], cv=kf, scoring='f1_macro')
lr_scores_cvss_score  = cross_val_score(lr, x, y['cvss_score'], cv=kf, scoring='f1_macro')

print("Logistic Regression Accuracy for 'type':", lr_accuracy_type)
print("Logistic Regression Accuracy for 'cvss_score' :", lr_accuracy_cvss_score)

print("K-Fold mean F1 (type):", lr_scores_type.mean())
print("K-Fold std  F1 (type):", lr_scores_type.std())

print("K-Fold mean F1 (cvss_score):", lr_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", lr_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the nu

Logistic Regression Accuracy for 'type': 0.36650485436893204
Logistic Regression Accuracy for 'cvss_score' : 0.5373786407766991
K-Fold mean F1 (type): 0.08376498810155399
K-Fold std  F1 (type): 0.0033207295646636107
K-Fold mean F1 (cvss_score): 0.23475904634961933
K-Fold std  F1 (cvss_score): 0.007096396172015192

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        28
         1.0       0.00      0.00      0.00        50
         2.0       0.00      0.00      0.00       103
         3.0       0.06      0.04      0.05       200
         4.0       0.40      0.79      0.53       834
         5.0       0.00      0.00      0.00        41
         6.0       0.00      0.00      0.00        36
         7.0       0.00      0.00      0.00       155
         8.0       0.30      0.05      0.09       176
         9.0       0.00      0.00      0.00        14
        10.0       0.32      0.19      0.24       423


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Decision Tree

In [79]:
dt = DecisionTreeClassifier(random_state=42)
multi_dt = MultiOutputClassifier(dt)

multi_dt.fit(x_train, y_train)
y_pred = multi_dt.predict(x_test)

dt_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
dt_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

dt_scores_type = cross_val_score(dt, x , y['type'], cv=kf, scoring='f1_macro')
dt_scores_cvss_score  = cross_val_score(dt, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Gradient Boosting Accuracy for 'type':", dt_accuracy_type)
print("Gradient Boosting Accuracy for 'cvss_score' :", dt_accuracy_cvss_score)

print("K-Fold mean F1 (type):", dt_scores_type.mean())
print("K-Fold std  F1 (type):", dt_scores_type.std())

print("K-Fold mean F1 (cvss_score):", dt_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", dt_scores_cvss_score.std())


print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Gradient Boosting Accuracy for 'type': 0.8063106796116505
Gradient Boosting Accuracy for 'cvss_score' : 0.5844660194174758
K-Fold mean F1 (type): 0.7243119835518702
K-Fold std  F1 (type): 0.0037779433085536537
K-Fold mean F1 (cvss_score): 0.45656365815339933
K-Fold std  F1 (cvss_score): 0.006227443254244629

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.81      0.79      0.80        28
         1.0       0.70      0.78      0.74        50
         2.0       0.40      0.47      0.43       103
         3.0       0.82      0.83      0.83       200
         4.0       0.84      0.81      0.83       834
         5.0       0.74      0.78      0.76        41
         6.0       0.49      0.50      0.49        36
         7.0       0.50      0.53      0.51       155
         8.0       0.95      0.96      0.96       176
         9.0       0.93      1.00      0.97        14
        10.0       0.95      0.93      0.94       423

    a

# RandomForest

In [80]:
rf = RandomForestClassifier(random_state=42)
multi_rf = MultiOutputClassifier(rf)

multi_rf.fit(x_train, y_train)
y_pred = multi_rf.predict(x_test)

rf_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
rf_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

rf_scores_type = cross_val_score(rf, x , y['type'], cv=kf, scoring='f1_macro')
rf_scores_cvss_score  = cross_val_score(rf, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Gradient Boosting Accuracy for 'type':", rf_accuracy_type)
print("Gradient Boosting Accuracy for 'cvss_score' :", rf_accuracy_cvss_score)

print("K-Fold mean F1 (type):", rf_scores_type.mean())
print("K-Fold std  F1 (type):", rf_scores_type.std())

print("K-Fold mean F1 (cvss_score):", rf_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", rf_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Gradient Boosting Accuracy for 'type': 0.8354368932038835
Gradient Boosting Accuracy for 'cvss_score' : 0.6286407766990292
K-Fold mean F1 (type): 0.669436773865622
K-Fold std  F1 (type): 0.008765196181456427
K-Fold mean F1 (cvss_score): 0.4960713737041411
K-Fold std  F1 (cvss_score): 0.014303723378917456

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.29      0.44        28
         1.0       0.80      0.70      0.74        50
         2.0       0.71      0.38      0.49       103
         3.0       0.74      0.88      0.80       200
         4.0       0.80      0.95      0.87       834
         5.0       0.84      0.63      0.72        41
         6.0       0.88      0.19      0.32        36
         7.0       0.77      0.39      0.52       155
         8.0       0.94      0.95      0.95       176
         9.0       1.00      0.50      0.67        14
        10.0       0.95      0.96      0.96       423

    accu

# Gradient Boosting

In [81]:
gb = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42)
multi_gbr = MultiOutputClassifier(gb)

multi_gbr.fit(x_train, y_train)
y_pred = multi_gbr.predict(x_test )

gb_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
gb_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

gb_scores_type = cross_val_score(gb, x , y['type'], cv=kf, scoring='f1_macro')
gb_scores_cvss_score  = cross_val_score(gb, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Gradient Boosting Accuracy for 'type':", gb_accuracy_type)
print("Gradient Boosting Accuracy for 'cvss_score' :", gb_accuracy_cvss_score)

print("K-Fold mean F1 (type):", gb_scores_type.mean())
print("K-Fold std  F1 (type):", gb_scores_type.std())

print("K-Fold mean F1 (cvss_score):", gb_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", gb_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Gradient Boosting Accuracy for 'type': 0.8475728155339806
Gradient Boosting Accuracy for 'cvss_score' : 0.6014563106796117
K-Fold mean F1 (type): 0.7631757854678933
K-Fold std  F1 (type): 0.0072637667710975075
K-Fold mean F1 (cvss_score): 0.42026741909962545
K-Fold std  F1 (cvss_score): 0.010788339491158684

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.76      0.79      0.77        28
         1.0       0.67      0.80      0.73        50
         2.0       0.76      0.18      0.30       103
         3.0       0.91      0.88      0.90       200
         4.0       0.80      0.95      0.87       834
         5.0       0.80      0.90      0.85        41
         6.0       0.79      0.53      0.63        36
         7.0       0.86      0.35      0.50       155
         8.0       0.96      0.98      0.97       176
         9.0       0.93      1.00      0.97        14
        10.0       0.93      0.95      0.94       423

    a

# Extra Tree

In [82]:
et = ExtraTreesClassifier(random_state=42)
multi_et = MultiOutputClassifier(et)

multi_et.fit(x_train, y_train)
y_pred = multi_et.predict(x_test )

et_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
et_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

et_scores_type = cross_val_score(et, x , y['type'], cv=kf, scoring='f1_macro')
et_scores_cvss_score  = cross_val_score(et, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Extra Trees Accuracy for 'type':", et_accuracy_type)
print("Extra Trees Accuracy for 'cvss_score' :", et_accuracy_cvss_score)

print("K-Fold mean F1 (type):", et_scores_type.mean())
print("K-Fold std  F1 (type):", et_scores_type.std())

print("K-Fold mean F1 (cvss_score):", et_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", et_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Extra Trees Accuracy for 'type': 0.8189320388349515
Extra Trees Accuracy for 'cvss_score' : 0.6077669902912621
K-Fold mean F1 (type): 0.6422477770245637
K-Fold std  F1 (type): 0.0216231975689129
K-Fold mean F1 (cvss_score): 0.4442371529175239
K-Fold std  F1 (cvss_score): 0.012618745880562178

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.83      0.18      0.29        28
         1.0       0.71      0.70      0.71        50
         2.0       0.66      0.39      0.49       103
         3.0       0.72      0.84      0.77       200
         4.0       0.80      0.93      0.86       834
         5.0       0.75      0.59      0.66        41
         6.0       0.78      0.19      0.31        36
         7.0       0.76      0.44      0.56       155
         8.0       0.94      0.90      0.92       176
         9.0       1.00      0.50      0.67        14
        10.0       0.93      0.93      0.93       423

    accuracy         

# Hist Gradient Boosting

In [83]:
hgb = HistGradientBoostingClassifier(max_iter=220, max_depth=5, random_state=42)
multi_hgb = MultiOutputClassifier(hgb)

multi_hgb.fit(x_train, y_train)
y_pred = multi_hgb.predict(x_test )

hgb_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
hgb_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

hgb_scores_type = cross_val_score(hgb, x , y['type'], cv=kf, scoring='f1_macro')
hgb_scores_cvss_score  = cross_val_score(hgb, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("HistGradientBoosting Accuracy for 'type':", hgb_accuracy_type)
print("HistGradientBoosting Accuracy for 'cvss_score' :", hgb_accuracy_cvss_score)

print("K-Fold mean F1 (type):", hgb_scores_type.mean())
print("K-Fold std  F1 (type):", hgb_scores_type.std())

print("K-Fold mean F1 (cvss_score):", hgb_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", hgb_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

HistGradientBoosting Accuracy for 'type': 0.3864077669902913
HistGradientBoosting Accuracy for 'cvss_score' : 0.6315533980582524
K-Fold mean F1 (type): 0.5434096370006727
K-Fold std  F1 (type): 0.3130459824121466
K-Fold mean F1 (cvss_score): 0.5129551889768581
K-Fold std  F1 (cvss_score): 0.013877171229916484

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.05      0.04      0.04        28
         1.0       0.18      0.24      0.20        50
         2.0       0.13      0.21      0.16       103
         3.0       0.43      0.42      0.43       200
         4.0       0.43      0.68      0.53       834
         5.0       0.29      0.41      0.34        41
         6.0       0.17      0.08      0.11        36
         7.0       0.22      0.16      0.19       155
         8.0       0.00      0.00      0.00       176
         9.0       0.00      0.00      0.00        14
        10.0       0.91      0.15      0.26       423

   

# KNN

In [84]:
knn = KNeighborsClassifier()
multi_knn = MultiOutputClassifier(knn)

multi_knn.fit(x_train, y_train)
y_pred = multi_knn.predict(x_test )

knn_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
knn_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

knn_scores_type = cross_val_score(knn, x , y['type'], cv=kf, scoring='f1_macro')
knn_scores_cvss_score  = cross_val_score(knn, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("KNN Accuracy for 'type':", knn_accuracy_type)
print("KNN Accuracy for 'cvss_score' :", knn_accuracy_cvss_score)

print("K-Fold mean F1 (type):", knn_scores_type.mean())
print("K-Fold std  F1 (type):", knn_scores_type.std())

print("K-Fold mean F1 (cvss_score):", knn_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", knn_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

KNN Accuracy for 'type': 0.5684466019417476
KNN Accuracy for 'cvss_score' : 0.512621359223301
K-Fold mean F1 (type): 0.330016286586144
K-Fold std  F1 (type): 0.010084356247648993
K-Fold mean F1 (cvss_score): 0.36856246764355155
K-Fold std  F1 (cvss_score): 0.007590915335924097

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.06      0.04      0.04        28
         1.0       0.32      0.44      0.37        50
         2.0       0.31      0.31      0.31       103
         3.0       0.52      0.65      0.58       200
         4.0       0.65      0.73      0.69       834
         5.0       0.14      0.10      0.11        41
         6.0       0.08      0.03      0.04        36
         7.0       0.43      0.19      0.26       155
         8.0       0.54      0.50      0.52       176
         9.0       0.00      0.00      0.00        14
        10.0       0.64      0.61      0.62       423

    accuracy                        

# Adaboost

In [85]:
ab = AdaBoostClassifier(n_estimators=220, random_state=42)
multi_ada = MultiOutputClassifier(ab)

multi_ada.fit(x_train, y_train)
y_pred = multi_ada.predict(x_test )

ab_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
ab_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

ab_scores_type = cross_val_score(ab, x , y['type'], cv=kf, scoring='f1_macro')
ab_scores_cvss_score  = cross_val_score(ab, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("AdaBoost Accuracy for 'type':", ab_accuracy_type)
print("AdaBoost Accuracy for 'cvss_score' :", ab_accuracy_cvss_score)

print("K-Fold mean F1 (type):", ab_scores_type.mean())
print("K-Fold std  F1 (type):", ab_scores_type.std())

print("K-Fold mean F1 (cvss_score):", ab_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", ab_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

AdaBoost Accuracy for 'type': 0.5713592233009709
AdaBoost Accuracy for 'cvss_score' : 0.566990291262136
K-Fold mean F1 (type): 0.32742661050711813
K-Fold std  F1 (type): 0.0406210086984585
K-Fold mean F1 (cvss_score): 0.27226543362069083
K-Fold std  F1 (cvss_score): 0.00583889772768262

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       1.00      0.11      0.19        28
         1.0       0.75      0.12      0.21        50
         2.0       0.00      0.00      0.00       103
         3.0       0.65      0.76      0.70       200
         4.0       0.54      0.89      0.67       834
         5.0       0.10      0.17      0.12        41
         6.0       0.00      0.00      0.00        36
         7.0       0.00      0.00      0.00       155
         8.0       0.30      0.11      0.17       176
         9.0       0.00      0.00      0.00        14
        10.0       0.87      0.58      0.69       423

    accuracy               

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classif

# Bagging

In [86]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(n_estimators=100, random_state=42)
multi_bagging = MultiOutputClassifier(bagging)

multi_bagging.fit(x_train, y_train)
y_pred = multi_bagging.predict(x_test )

bag_accuracy_type = accuracy_score(y_test['type'], y_pred[:, 0])
bag_accuracy_cvss_score  = accuracy_score(y_test['cvss_score'], y_pred[:, 1])

bag_scores_type = cross_val_score(bagging, x , y['type'], cv=kf, scoring='f1_macro')
bag_scores_cvss_score  = cross_val_score(bagging, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging Accuracy for 'type':", bag_accuracy_type)
print("Bagging Accuracy for 'cvss_score' :", bag_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_scores_type.mean())
print("K-Fold std  F1 (type):", bag_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging Accuracy for 'type': 0.8679611650485437
Bagging Accuracy for 'cvss_score' : 0.6223300970873786
K-Fold mean F1 (type): 0.7838282165585465
K-Fold std  F1 (type): 0.007316027792375406
K-Fold mean F1 (cvss_score): 0.5156276774678378
K-Fold std  F1 (cvss_score): 0.010810655997815114

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.91      0.75      0.82        28
         1.0       0.75      0.80      0.78        50
         2.0       0.69      0.50      0.58       103
         3.0       0.89      0.87      0.88       200
         4.0       0.84      0.94      0.89       834
         5.0       0.86      0.88      0.87        41
         6.0       0.82      0.50      0.62        36
         7.0       0.72      0.47      0.57       155
         8.0       0.97      0.98      0.97       176
         9.0       1.00      1.00      1.00        14
        10.0       0.96      0.96      0.96       423

    accuracy               

# Hard Voting

In [87]:
from sklearn.ensemble import VotingClassifier

model1 = RandomForestClassifier(n_estimators=200, random_state=42)
model2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
model3 = LogisticRegression(max_iter=500)

voting_hard = VotingClassifier(
    estimators=[
        ('rf', model1),
        ('et', model2),
        ('lr', model3)
    ],
    voting='hard'
)

multi_voting_hard = MultiOutputClassifier(voting_hard)

multi_voting_hard.fit(x_train, y_train)
y_pred = multi_voting_hard.predict(x_test)

hard_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
hard_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])


hard_scores_type = cross_val_score(voting_hard, x, y['type'], cv=kf, scoring='f1_macro')
hard_scores_cvss_score  = cross_val_score(voting_hard, x, y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Hard Voting Accuracy for type:", hard_acc_type)
print("Hard Voting Accuracy for cvss_score :", hard_acc_cvss_score)

print("K-fold F1 mean (type):", hard_scores_type.mean())
print("K-fold F1 std  (type):", hard_scores_type.std())

print("K-fold F1 mean (cvss_score) :", hard_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", hard_scores_cvss_score.std())

print(classification_report(y_test["type"], y_pred[:, 0]))

Hard Voting Accuracy for type: 0.8242718446601942
Hard Voting Accuracy for cvss_score : 0.6174757281553398
K-fold F1 mean (type): 0.6515998759084775
K-fold F1 std  (type): 0.015156760433302432
K-fold F1 mean (cvss_score) : 0.4503310811762777
K-fold F1 std  (cvss_score) : 0.013775987678280118
              precision    recall  f1-score   support

         0.0       1.00      0.32      0.49        28
         1.0       0.76      0.70      0.73        50
         2.0       0.67      0.35      0.46       103
         3.0       0.73      0.86      0.79       200
         4.0       0.78      0.95      0.86       834
         5.0       0.84      0.63      0.72        41
         6.0       1.00      0.17      0.29        36
         7.0       0.80      0.37      0.50       155
         8.0       0.95      0.90      0.92       176
         9.0       1.00      0.36      0.53        14
        10.0       0.95      0.94      0.95       423

    accuracy                           0.82      2060
   

# Soft Voting

In [88]:
model1 = RandomForestClassifier(n_estimators=200, random_state=42)
model2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
model3 = LogisticRegression(max_iter=500)

voting_hard = VotingClassifier(
    estimators=[
        ('rf', model1),
        ('et', model2),
        ('lr', model3)
    ],
    voting='soft'
)

multi_voting_hard = MultiOutputClassifier(voting_hard)

multi_voting_hard.fit(x_train, y_train)
y_pred = multi_voting_hard.predict(x_test)

soft_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
soft_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

soft_scores_type = cross_val_score(voting_hard, x, y['type'], cv=kf, scoring='f1_macro')
soft_scores_cvss_score  = cross_val_score(voting_hard, x, y['cvss_score'],  cv=kf, scoring='f1_macro')


print("Hard Voting Accuracy for type:", soft_acc_type)
print("Hard Voting Accuracy for cvss_score :", soft_acc_cvss_score)

print("K-fold F1 mean (type):", soft_scores_type.mean())
print("K-fold F1 std  (type):", soft_scores_type.std())

print("K-fold F1 mean (cvss_score) :", soft_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score) :", soft_scores_cvss_score.std())

print(classification_report(y_test["type"], y_pred[:, 0]))

Hard Voting Accuracy for type: 0.8203883495145631
Hard Voting Accuracy for cvss_score : 0.6257281553398059
K-fold F1 mean (type): 0.6132945415392504
K-fold F1 std  (type): 0.01750175151507526
K-fold F1 mean (cvss_score) : 0.4169114820496899
K-fold F1 std  (cvss_score) : 0.0161809637880913
              precision    recall  f1-score   support

         0.0       1.00      0.18      0.30        28
         1.0       0.80      0.66      0.73        50
         2.0       0.80      0.34      0.48       103
         3.0       0.74      0.88      0.80       200
         4.0       0.76      0.96      0.85       834
         5.0       0.89      0.59      0.71        41
         6.0       1.00      0.14      0.24        36
         7.0       0.83      0.32      0.46       155
         8.0       0.95      0.88      0.91       176
         9.0       1.00      0.43      0.60        14
        10.0       0.95      0.94      0.95       423

    accuracy                           0.82      2060
   mac

# Stacking

In [89]:
from sklearn.ensemble import StackingClassifier

base1 = RandomForestClassifier(n_estimators=200, random_state=42)
base2 = ExtraTreesClassifier(n_estimators=200, random_state=42)
base3 = LogisticRegression(max_iter=500)

stacking = StackingClassifier(
    estimators=[
        ('rf', base1),
        ('et', base2),
        ('lr', base3)
    ],
    final_estimator=LogisticRegression(max_iter=500)
)

multi_stacking = MultiOutputClassifier(stacking)

multi_stacking.fit(x_train, y_train)
y_pred = multi_stacking.predict(x_test)

stacking_acc_type = accuracy_score(y_test["type"], y_pred[:, 0])
stacking_acc_cvss_score  = accuracy_score(y_test["cvss_score"],  y_pred[:, 1])

stacking_scores_type = cross_val_score(stacking, x, y['type'], cv=kf, scoring='f1_macro')
stacking_scores_cvss_score  = cross_val_score(stacking, x, y['cvss_score'],  cv=kf, scoring='f1_macro')

print("Stacking Accuracy for type:", stacking_acc_type)
print("Stacking Accuracy for cvss_score :", stacking_acc_cvss_score)

print("K-fold F1 mean (type):", stacking_scores_type.mean())
print("K-fold F1 std  (type):", stacking_scores_type.std())

print("K-fold F1 mean (cvss_score):", stacking_scores_cvss_score.mean())
print("K-fold F1 std  (cvss_score):", stacking_scores_cvss_score.std())

print(classification_report(y_test["type"], y_pred[:, 0]))

Stacking Accuracy for type: 0.8446601941747572
Stacking Accuracy for cvss_score : 0.6368932038834951
K-fold F1 mean (type): 0.6972474531579157
K-fold F1 std  (type): 0.014847204394619819
K-fold F1 mean (cvss_score): 0.4949531750271971
K-fold F1 std  (cvss_score): 0.013024863581860117
              precision    recall  f1-score   support

         0.0       1.00      0.36      0.53        28
         1.0       0.81      0.68      0.74        50
         2.0       0.68      0.38      0.49       103
         3.0       0.80      0.86      0.83       200
         4.0       0.81      0.95      0.87       834
         5.0       0.80      0.80      0.80        41
         6.0       0.80      0.22      0.35        36
         7.0       0.76      0.44      0.56       155
         8.0       0.95      0.95      0.95       176
         9.0       0.90      0.64      0.75        14
        10.0       0.95      0.96      0.95       423

    accuracy                           0.84      2060
   macro av

# Bagged KNN

In [90]:
knn = KNeighborsClassifier()
bag_knn = BaggingClassifier(estimator=knn, n_estimators=100, random_state=42)
multi_bag_knn = MultiOutputClassifier(bag_knn)

multi_bag_knn.fit(x_train, y_train)
y_pred = multi_bag_knn.predict(x_test )

bag_knn_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
bag_knn_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], y_pred[:,1])

bag_knn_scores_type = cross_val_score(bag_knn, x , y['type'], cv=kf, scoring='f1_macro')
bag_knn_scores_cvss_score = cross_val_score(bag_knn, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging KNN Accuracy for 'type':", bag_knn_accuracy_type)
print("Bagging KNN Accuracy for 'cvss_score':", bag_knn_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_knn_scores_type.mean())
print("K-Fold std  F1 (type):", bag_knn_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_knn_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_knn_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging KNN Accuracy for 'type': 0.5864077669902913
Bagging KNN Accuracy for 'cvss_score': 0.5485436893203883
K-Fold mean F1 (type): 0.33302250573585085
K-Fold std  F1 (type): 0.012939165059061506
K-Fold mean F1 (cvss_score): 0.3532474713754801
K-Fold std  F1 (cvss_score): 0.0077456764758699415

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.33      0.04      0.06        28
         1.0       0.42      0.50      0.46        50
         2.0       0.42      0.28      0.34       103
         3.0       0.54      0.65      0.59       200
         4.0       0.66      0.73      0.69       834
         5.0       0.14      0.12      0.13        41
         6.0       0.08      0.03      0.04        36
         7.0       0.40      0.20      0.27       155
         8.0       0.54      0.51      0.53       176
         9.0       0.33      0.07      0.12        14
        10.0       0.61      0.67      0.64       423

    accuracy      

# Bagged DecisionTreeClassifier

In [91]:
dt = DecisionTreeClassifier(random_state=42)
bag_dt = BaggingClassifier(estimator=dt, n_estimators=100, random_state=42)
multi_bag_dt = MultiOutputClassifier(bag_dt)

multi_bag_dt.fit(x_train, y_train)
y_pred = multi_bag_dt.predict(x_test)

bag_dt_accuracy_type = accuracy_score(y_test['type'], y_pred[:,0])
bag_dt_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], y_pred[:,1])

bag_dt_scores_type = cross_val_score(bag_dt, x , y['type'], cv=kf, scoring='f1_macro')
bag_dt_scores_cvss_score = cross_val_score(bag_dt, x , y['cvss_score'], cv=kf, scoring='f1_macro')

print("Bagging Decision Tree Accuracy for 'type':", bag_dt_accuracy_type)
print("Bagging Decision Tree Accuracy for 'cvss_score':", bag_dt_accuracy_cvss_score)

print("K-Fold mean F1 (type):", bag_dt_scores_type.mean())
print("K-Fold std  F1 (type):", bag_dt_scores_type.std())

print("K-Fold mean F1 (cvss_score):", bag_dt_scores_cvss_score.mean())
print("K-Fold std  F1 (cvss_score):", bag_dt_scores_cvss_score.std())

print("\nClassification Report for 'type':\n", classification_report(y_test['type'], y_pred[:,0]))
print("\nClassification Report for 'cvss_score':\n", classification_report(y_test['cvss_score'], y_pred[:,1]))

Bagging Decision Tree Accuracy for 'type': 0.8679611650485437
Bagging Decision Tree Accuracy for 'cvss_score': 0.6223300970873786
K-Fold mean F1 (type): 0.7838282165585465
K-Fold std  F1 (type): 0.007316027792375406
K-Fold mean F1 (cvss_score): 0.5156276774678378
K-Fold std  F1 (cvss_score): 0.010810655997815114

Classification Report for 'type':
               precision    recall  f1-score   support

         0.0       0.91      0.75      0.82        28
         1.0       0.75      0.80      0.78        50
         2.0       0.69      0.50      0.58       103
         3.0       0.89      0.87      0.88       200
         4.0       0.84      0.94      0.89       834
         5.0       0.86      0.88      0.87        41
         6.0       0.82      0.50      0.62        36
         7.0       0.72      0.47      0.57       155
         8.0       0.97      0.98      0.97       176
         9.0       1.00      1.00      1.00        14
        10.0       0.96      0.96      0.96       423



In [92]:
from rich.table import Table
from rich.console import Console

console = Console()

results = [
    ['LogisticRegression', lr_accuracy_type, lr_scores_type.mean(), lr_scores_type.std(), lr_accuracy_cvss_score, lr_scores_cvss_score.mean(), lr_scores_cvss_score.std()],
    ['DecisionTree', dt_accuracy_type, dt_scores_type.mean(), dt_scores_type.std(), dt_accuracy_cvss_score, dt_scores_cvss_score.mean(), dt_scores_cvss_score.std()],
    ['RandomForest', rf_accuracy_type, rf_scores_type.mean(), rf_scores_type.std(), rf_accuracy_cvss_score, rf_scores_cvss_score.mean(), rf_scores_cvss_score.std()],
    ['ExtraTrees', et_accuracy_type, et_scores_type.mean(), et_scores_type.std(), et_accuracy_cvss_score, et_scores_cvss_score.mean(), et_scores_cvss_score.std()],
    ['GradientBoosting', gb_accuracy_type, gb_scores_type.mean(), gb_scores_type.std(), gb_accuracy_cvss_score, gb_scores_cvss_score.mean(), gb_scores_cvss_score.std()],
    ['HistGradientBoosting', hgb_accuracy_type, hgb_scores_type.mean(), hgb_scores_type.std(), hgb_accuracy_cvss_score, hgb_scores_cvss_score.mean(), hgb_scores_cvss_score.std()],
    ['KNN', knn_accuracy_type, knn_scores_type.mean(), knn_scores_type.std(), knn_accuracy_cvss_score, knn_scores_cvss_score.mean(), knn_scores_cvss_score.std()],
    ['AdaBoost', ab_accuracy_type, ab_scores_type.mean(), ab_scores_type.std(), ab_accuracy_cvss_score, ab_scores_cvss_score.mean(), ab_scores_cvss_score.std()],
    ['Bagging', bag_accuracy_type, bag_scores_type.mean(), bag_scores_type.std(), bag_accuracy_cvss_score, bag_scores_cvss_score.mean(), bag_scores_cvss_score.std()],
    ['Hard Voting', hard_acc_type, hard_scores_type.mean(), hard_scores_type.std(), hard_acc_cvss_score, hard_scores_cvss_score.mean(), hard_scores_cvss_score.std()],
    ['Soft Voting', soft_acc_type, soft_scores_type.mean(), soft_scores_type.std(), soft_acc_cvss_score, soft_scores_cvss_score.mean(), soft_scores_cvss_score.std()],
    ['Stacking', stacking_acc_type, stacking_scores_type.mean(), stacking_scores_type.std(), stacking_acc_cvss_score, stacking_scores_cvss_score.mean(), stacking_scores_cvss_score.std()],
    ['Bagged KNN', bag_knn_accuracy_type, bag_knn_scores_type.mean(), bag_knn_scores_type.std(), bag_knn_accuracy_cvss_score, bag_knn_scores_cvss_score.mean(), bag_knn_scores_cvss_score.std()],
    ['Bagged DT', bag_dt_accuracy_type, bag_dt_scores_type.mean(), bag_dt_scores_type.std(), bag_dt_accuracy_cvss_score, bag_dt_scores_cvss_score.mean(), bag_dt_scores_cvss_score.std()],
]


for row in results:
    type_acc = row[1]
    cvss_score_acc = row[4]
    combined = (type_acc + cvss_score_acc) / 2
    row.append(combined)

result_sorted = sorted(results, key=lambda i: i[-1], reverse=True)

best_model = max(results, key=lambda x: x[-1])
worst_model = min(results, key=lambda x: x[-1])

table = Table(title="Filter Method Comparison", show_lines=True)
table.add_column("Algorithm")
table.add_column("Type Acc")
table.add_column("K-Fold Mean")
table.add_column("K-Fold Std")
table.add_column("cvss_score Acc")
table.add_column("K-Fold Mean")
table.add_column("K-Fold Std")
table.add_column("Combined", justify="right")

for row in result_sorted:
    algo, type_acc, kmean_type, kstd_type, cvss_score_acc, kmean_cvss_score, kstd_cvss_score, combined = row

    if row == best_model:
        table.add_row(
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{type_acc:.2f}[/bold green]",
            f"[bold green]{kmean_type:.2f}[/bold green]",
            f"[bold green]{kstd_type:.2f}[/bold green]",
            f"[bold green]{cvss_score_acc:.2f}[/bold green]",
            f"[bold green]{kmean_cvss_score:.2f}[/bold green]",
            f"[bold green]{kstd_cvss_score:.2f}[/bold green]",
            f"[bold green]{combined:.2f}[/bold green]",
        )
    elif row == worst_model:
        table.add_row(
            f"[bold red]{algo}[/bold red]",
            f"[bold red]{type_acc:.2f}[/bold red]",
            f"[bold red]{kmean_type:.2f}[/bold red]",
            f"[bold red]{kstd_type:.2f}[/bold red]",
            f"[bold red]{cvss_score_acc:.2f}[/bold red]",
            f"[bold red]{kmean_cvss_score:.2f}[/bold red]",
            f"[bold red]{kstd_cvss_score:.2f}[/bold red]",
            f"[bold red]{combined:.2f}[/bold red]",
        )
    else:
        table.add_row(
            algo, f"{type_acc:.2f}", f"{kmean_type:.2f}", f"{kstd_type:.2f}",
            f"{cvss_score_acc:.2f}", f"{kmean_cvss_score:.2f}", f"{kstd_cvss_score:.2f}", f"{combined:.2f}"
        )

console.print(table)


                                             Filter Method Comparison                                              
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Algorithm          ┃ Type Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ cvss_score Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ Combined ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Bagging            │ 0.87     │ 0.78        │ 0.01       │ 0.62           │ 0.52        │ 0.01       │     0.75 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged DT          │ 0.87     │ 0.78        │ 0.01       │ 0.62           │ 0.52        │ 0.01       │     0.75 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Stacking           │ 0.84     │ 0.70        │ 0.01       │ 0.64           │ 0.49        │ 0.01       │     0.74 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ RandomForest       │ 0.84     │ 0.67        │ 0.01       │ 0.63           │ 0.50        │ 0.01       │     0.73 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ GradientBoosting   │ 0.85     │ 0.76        │ 0.01       │ 0.60           │ 0.42        │ 0.01       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Soft Voting        │ 0.82     │ 0.61        │ 0.02       │ 0.63           │ 0.42        │ 0.02       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Hard Voting        │ 0.82     │ 0.65        │ 0.02       │ 0.62           │ 0.45        │ 0.01       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ ExtraTrees         │ 0.82     │ 0.64        │ 0.02       │ 0.61           │ 0.44        │ 0.01       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ DecisionTree       │ 0.81     │ 0.72        │ 0.00       │ 0.58           │ 0.46        │ 0.01       │     0.70 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ AdaBoost           │ 0.57     │ 0.33        │ 0.04       │ 0.57           │ 0.27        │ 0.01       │     0.57 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged KNN         │ 0.59     │ 0.33        │ 0.01       │ 0.55           │ 0.35        │ 0.01       │     0.57 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ KNN                │ 0.57     │ 0.33        │ 0.01       │ 0.51           │ 0.37        │ 0.01       │     0.54 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ HistGradientBoost… │ 0.39     │ 0.54        │ 0.31       │ 0.63           │ 0.51        │ 0.01       │     0.51 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LogisticRegression │ 0.37     │ 0.08        │ 0.00       │ 0.54           │ 0.23        │ 0.01       │     0.45 │
└────────────────────┴──────────┴─────────────┴────────────┴────────────────┴─────────────┴────────────┴──────────┘

In [93]:
os.makedirs('results', exist_ok=True)

temp_console = Console(record=True)
temp_console.print(table)
text = temp_console.export_text()
with open('results/baseline_model.txt', 'a', encoding='utf-8') as f:
    f.write(text)

                                             Filter Method Comparison                                              
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Algorithm          ┃ Type Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ cvss_score Acc ┃ K-Fold Mean ┃ K-Fold Std ┃ Combined ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Bagging            │ 0.87     │ 0.78        │ 0.01       │ 0.62           │ 0.52        │ 0.01       │     0.75 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged DT          │ 0.87     │ 0.78        │ 0.01       │ 0.62           │ 0.52        │ 0.01       │     0.75 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Stacking           │ 0.84     │ 0.70        │ 0.01       │ 0.64           │ 0.49        │ 0.01       │     0.74 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ RandomForest       │ 0.84     │ 0.67        │ 0.01       │ 0.63           │ 0.50        │ 0.01       │     0.73 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ GradientBoosting   │ 0.85     │ 0.76        │ 0.01       │ 0.60           │ 0.42        │ 0.01       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Soft Voting        │ 0.82     │ 0.61        │ 0.02       │ 0.63           │ 0.42        │ 0.02       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Hard Voting        │ 0.82     │ 0.65        │ 0.02       │ 0.62           │ 0.45        │ 0.01       │     0.72 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ ExtraTrees         │ 0.82     │ 0.64        │ 0.02       │ 0.61           │ 0.44        │ 0.01       │     0.71 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ DecisionTree       │ 0.81     │ 0.72        │ 0.00       │ 0.58           │ 0.46        │ 0.01       │     0.70 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ AdaBoost           │ 0.57     │ 0.33        │ 0.04       │ 0.57           │ 0.27        │ 0.01       │     0.57 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ Bagged KNN         │ 0.59     │ 0.33        │ 0.01       │ 0.55           │ 0.35        │ 0.01       │     0.57 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ KNN                │ 0.57     │ 0.33        │ 0.01       │ 0.51           │ 0.37        │ 0.01       │     0.54 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ HistGradientBoost… │ 0.39     │ 0.54        │ 0.31       │ 0.63           │ 0.51        │ 0.01       │     0.51 │
├────────────────────┼──────────┼─────────────┼────────────┼────────────────┼─────────────┼────────────┼──────────┤
│ LogisticRegression │ 0.37     │ 0.08        │ 0.00       │ 0.54           │ 0.23        │ 0.01       │     0.45 │
└────────────────────┴──────────┴─────────────┴────────────┴────────────────┴─────────────┴────────────┴──────────┘